In [1]:
import numpy as np
import pandas as pd

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [4]:
filepath = "../data/Final Categorization Data.csv"
df = pd.read_csv(filepath)

#drop extra binned columns for categorization models
df = df.drop(df.iloc[:,82:100], axis = 1)
df['Origin Destination Province'] = df['Origin Province'] + df['Destination Province']
df['Origin Destination Region'] = df['Origin Province'] + df['Destination Region']
df['Origin Destination New Region'] = df['Origin Province'] + df['New Regions']
df['Origin Destination Metro Area'] = df['Origin Province'] + df['Destination Metro Area']

#one hot encode remaining categorical variables
one_hot_columns = ["Main Transportation Method", "Quarter", "Main Reason" ]
df2 = pd.get_dummies(df, columns = one_hot_columns)
#breaking into separate dataframes for diff origin destination field pairings made into one hot encodings 
#each has like 100s of new columns added 
# df2odp = pd.get_dummies(df2, columns = ["Origin Destination Province"])
# df2odr = pd.get_dummies(df2, columns = ["Origin Destination Region"])
# df2odnr = pd.get_dummies(df2, columns = ["Origin Destination New Region"])
# df2odma = pd.get_dummies(df2, columns = ["Origin Destination Metro Area"])

df2 = pd.get_dummies(df2, columns = ["Origin Destination Province"])
df2 = pd.get_dummies(df2, columns = ["Origin Destination Region"])
df2 = pd.get_dummies(df2, columns = ["Origin Destination New Region"])
df2 = pd.get_dummies(df2, columns = ["Origin Destination Metro Area"])

df2

,PUMFID,REFYEAR,TRIPID,TRIPTYPE,ROS_Q02A,ROS_Q02B,DISTANCE,DISTLONG,DURATION,DUR_MAIN,...,Origin Destination Metro Area_SaskatchewanSaskatoon,Origin Destination Metro Area_SaskatchewanSherbrooke,Origin Destination Metro Area_SaskatchewanSt-Catherines/Niagara,Origin Destination Metro Area_SaskatchewanSt. John's,Origin Destination Metro Area_SaskatchewanThunder Bay,Origin Destination Metro Area_SaskatchewanToronto,Origin Destination Metro Area_SaskatchewanUnknown,Origin Destination Metro Area_SaskatchewanVancouver,Origin Destination Metro Area_SaskatchewanVictoria,Origin Destination Metro Area_SaskatchewanWinnipeg
0,1006690,2022,2,5,2,96,46.0,46.0,0,0,...,False,False,False,False,False,False,False,False,False,False
1,1006690,2022,3,5,2,96,63.0,63.0,0,0,...,False,False,False,False,False,False,False,False,False,False
2,3008513,2022,1,4,2,96,847.0,847.0,76,76,...,False,False,False,False,False,False,False,False,False,False
3,1010819,2022,1,6,2,96,83.0,83.0,15,15,...,False,False,False,False,False,False,False,False,False,False
4,1006697,2022,2,5,2,96,NaN,NaN,0,0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47136,4034637,2022,2,5,2,96,180.0,180.0,0,0,...,False,False,False,False,False,False,False,False,False,False
47137,4034637,2022,4,5,1,1,42.0,42.0,0,0,...,False,False,False,False,False,False,False,False,False,False
47138,4034640,2022,1,5,1,7,101.0,101.0,0,0,...,False,False,False,False,False,False,False,False,False,False
47139,4034647,2022,1,5,2,96,112.0,112.0,0,0,...,False,False,False,False,False,False,False,False,False,False


In [5]:
#taking subsets of columns for easier grouping and error checking
transport_cols = [col for col in df2.columns if 'Main Transportation Method' in col]
quarter_cols = [col for col in df2.columns if 'Quarter' in col]
mainreason_cols = [col for col in df2.columns if 'Main Reason' in col]
Origin_Destination_Province_cols = [col for col in df2.columns if 'Origin Destination Province' in col]
Origin_Destination_Region_cols = [col for col in df2.columns if 'Origin Destination Region' in col]
Origin_Destination_New_Region_cols = [col for col in df2.columns if 'Origin Destination New Region' in col]
Origin_Destination_Metro_Area_cols = [col for col in df2.columns if 'Origin Destination Metro Area' in col]
Origin_Destination_New_Region_cols
#cat_features = ['Origin Province','Destination Region', 'Main Reason', 'Visit Friends & Family', "Restaurant/Bar/Club", 
#                "Shopping", "Sightseeing", "Museum/Art Gallery", "Historic site", "Zoo/Aquarium", "Wildife Viewing/Bird Watching", "National/Provincial/Nature Park"]

In [6]:
#from column lists taking subsets of dataframe for easier grouping later

# independent features
# num features -> duration, number of children, number of adults
num_features = df2.loc[:,["DURATION", "GNCQ06A", "GNCQ06B"]]
transport = df2.loc[:,transport_cols]
quarter = df2.loc[:,quarter_cols]
main_reason = df2.loc[:,mainreason_cols]
ori_dest_province = df2.loc[:,Origin_Destination_Province_cols]
ori_dest_region = df2.loc[:,Origin_Destination_Region_cols]
ori_dest_newregion = df2.loc[:,Origin_Destination_New_Region_cols]
ori_dest_metroarea = df2.loc[:,Origin_Destination_Metro_Area_cols]

In [7]:
# same as above but dependent features (costs)
total_spend = df2.loc[:,["SPD_DTOT"]]
transport_spend = df2.loc[:,["Total Spending for transport"]]
foodbev_spend = df2.loc[:,["Total Spending for food/bevs"]]
activities_spend = df2.loc[:,["Total spending for activities/entertainment"]]
shopping_spend = df2.loc[:,["Total spending on shopping"]]
accomodation_spend = df2.loc[:,["SPD_D06"]]


In [8]:
#subset of df columns for all activities
all_activities_cols = df2.iloc[:,45:79]

#based off high correlation activites as found in correlations notebook
high_correlation_activities_cols =  df2.loc[:,['Visit Friends & Family', "Restaurant/Bar/Club", "Shopping", "Sightseeing", "Museum/Art Gallery", "Historic site", "Zoo/Aquarium", "Wildife Viewing/Bird Watching", "National/Provincial/Nature Park"]]

In [10]:
#separate out features for model 1

#first set of dependent variables to test 
#consists of numerical inputs + ori dest province, transport, quarter, mainreason, high corr actvities
x1_list = [num_features, ori_dest_province, transport, quarter, main_reason, high_correlation_activities_cols]
x1_list_transposed = [list_item.transpose() for list_item in x1_list]
x1 = pd.concat(x1_list_transposed, axis=0).transpose()
y1 = total_spend
#check all row values coming in as non null
#x1.isnull().any().any()

#testing stuff
y1.mean()
dataframes_list = [x1, y1]
dataframes_list_transposed = [list_item.transpose() for list_item in dataframes_list]
merged_dataframe = pd.concat(dataframes_list_transposed, axis=0).transpose()
merged_dataframe.head()

,DURATION,GNCQ06A,GNCQ06B,Origin Destination Province_AlbertaAlberta,Origin Destination Province_AlbertaBritish Columbia,Origin Destination Province_AlbertaManitoba,Origin Destination Province_AlbertaNew Brunswick,Origin Destination Province_AlbertaNewfoundland and Labrador,Origin Destination Province_AlbertaNorthwest Territories,Origin Destination Province_AlbertaNova Scotia,...,Visit Friends & Family,Restaurant/Bar/Club,Shopping,Sightseeing,Museum/Art Gallery,Historic site,Zoo/Aquarium,Wildife Viewing/Bird Watching,National/Provincial/Nature Park,SPD_DTOT
0,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,40
1,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,30
2,76,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,15945
3,15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,15300
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,20


In [31]:
# feature set 1 - total spend

X_train, X_test, y_train, y_test = train_test_split(x1, y1, test_size=0.2, random_state=42)
model1 = LinearRegression(positive = True, fit_intercept=False).fit(X_train, y_train)

y_pred = model1.predict(X_test)
# mse_model = mean_squared_error(y_test, y_pred)
# mae_model = mean_absolute_error(y_test, y_pred)
# print(y_pred)
# print(y_test)
# print(f"Model - MSE: {mse_model} MAE: {mae_model}")

#snigle value test
testx = x1.iloc[510:520,:]
correcty = y1.iloc[510:520,:]
testy = model1.predict(testx)
print(f'predict {testy} when actually {correcty}') 
# print(mean_squared_error(testy, correcty))
# print((correcty - testy)**2)



#print(f'ydiff{y_pred - y_test}')
#print(f"Model1 - Mean Squared Error: {mse_model}")

predict [[1270.96046469]
 [ 297.67613567]
 [ 232.97947253]
 [ 158.18666498]
 [5425.08058048]
 [  72.1336976 ]
 [   0.        ]
 [ 344.08991845]
 [ 263.66703098]
 [ 263.66703098]] when actually      SPD_DTOT
510      5491
511       135
512       210
513         0
514      8624
515        75
516        80
517       190
518       275
519       316


In [174]:
kf = KFold(n_splits=10,random_state=42,shuffle=True)

cnt = 1
for train_index, test_index in kf.split(x1, y1):
   X_train, X_test, y_train, y_test = train_test_split(x1, y1)
   model1 = LinearRegression().fit(X_train, y_train)
   y_pred = model1.predict(X_test)
   mse_model1 = mean_squared_error(y_test, y_pred)
   # print("Mean Score: " + str(np.mean(scores)))
   print(f"Model 1 - Fold:{cnt} MSE: {mse_model1}")
   cnt += 1


# model1 = LinearRegression().fit(X_train, y_train)

# y_pred = model1.predict(X_test)
# mse_model1 = mean_squared_error(y_test, y_pred)
# print(f"Model1 - Mean Squared Error: {mse_model1}")

Model 1 - Fold:1 MSE: 4.874234814097397e+23
Model 1 - Fold:2 MSE: 3.445925015273702e+24
Model 1 - Fold:3 MSE: 1.4885613575156777e+24
Model 1 - Fold:4 MSE: 2.680901275059343e+24
Model 1 - Fold:5 MSE: 1.1905551028131713e+25
Model 1 - Fold:6 MSE: 4.141720467872141e+24
Model 1 - Fold:7 MSE: 3.3446573598765465e+23
Model 1 - Fold:8 MSE: 9.786865619848688e+23
Model 1 - Fold:9 MSE: 1.0398751694824983e+25
Model 1 - Fold:10 MSE: 6.063831630469364e+24


In [176]:
def rmse_model1(score):
    rmse_model1 = np.sqrt(-score)
    print(f'rmse= {"{:.2f}".format(rmse_model1)}')

score_tree = cross_val_score(model1, x1, y1, cv=kf, scoring="neg_mean_squared_error")

In [154]:
# feature set 2 - total spend


#consists of numerical inputs + ori dest transport, transport, quarter, mainreason, high corr activities
x2_list = [num_features, ori_dest_region, transport, quarter, main_reason, high_correlation_activities_cols]
x2_list_transposed = [list_item.transpose() for list_item in x2_list]
x2 = pd.concat(x2_list_transposed, axis=0).transpose()
y2 = total_spend

kf = KFold(n_splits=10,random_state=42,shuffle=True)

cnt = 1
for train_index, test_index in kf.split(x2, y2):
   X_train, X_test, y_train, y_test = train_test_split(x2, y2)
   model2 = LinearRegression().fit(X_train, y_train)
   y_pred = model2.predict(X_test)
   mse_model2 = mean_squared_error(y_test, y_pred)
   # print("Mean Score: " + str(np.mean(scores)))
   print(f"Model 2 - Fold:{cnt} MSE: {mse_model2}")
   cnt += 1


Model 2 - Fold:1 MSE: 2.3905887301223572e+24
Model 2 - Fold:2 MSE: 9.572491445314784e+22
Model 2 - Fold:3 MSE: 4.6400954843958704e+22
Model 2 - Fold:4 MSE: 3.93089533424881e+25
Model 2 - Fold:5 MSE: 7.296941590411135e+25
Model 2 - Fold:6 MSE: 3.3327085023446816e+23
Model 2 - Fold:7 MSE: 2.8130095646868783e+22
Model 2 - Fold:8 MSE: 8.272533559336589e+20
Model 2 - Fold:9 MSE: 3.949029438860416e+23
Model 2 - Fold:10 MSE: 1.8764468793219562e+23


In [116]:
# feature set 3 - total spend

#consists of numerical inputs + ori dest new region, transport, transport, quarter, mainreason, high corr activities
x3_list = [num_features, ori_dest_newregion, transport, quarter, main_reason, high_correlation_activities_cols]
# x3_list = [num_features, ori_dest_newregion, transport, quarter, main_reason, all_activities_cols]
x3_list_transposed = [list_item.transpose() for list_item in x3_list]
x3 = pd.concat(x3_list_transposed, axis=0).transpose()
y3 = total_spend

In [96]:
kf = KFold(n_splits=10,random_state=42,shuffle=True)

y_true_list = []
y_pred_list = []

cnt = 1
for train_index, test_index in kf.split(x3, y3):
   X_train, X_test, y_train, y_test = train_test_split(x3, y3)
   model3 = LinearRegression().fit(X_train, y_train)
   y_pred = model3.predict(X_test)
   y_true_list = np.append(y_true_list, y_test)
   y_pred_list = np.append(y_pred_list,y_pred)
   mse_model3 = mean_squared_error(y_test, y_pred)
   mae_model = mean_absolute_error(y_test, y_pred)
   # print("Mean Score: " + str(np.mean(scores)))
   print(f"Model 3 - Fold:{cnt} MSE: {mse_model3}  MAE: {mae_model}")
   cnt += 1


Model 3 - Fold:1 MSE: 4.0392551379975285e+21  MAE: 629287577.0529675
Model 3 - Fold:2 MSE: 1.083941189664417e+24  MAE: 16172809373.920658
Model 3 - Fold:3 MSE: 4.4683357765553015e+23  MAE: 9497247895.557526
Model 3 - Fold:4 MSE: 3.135314947479267e+24  MAE: 29036062777.898205
Model 3 - Fold:5 MSE: 10913720.540075459  MAE: 381.89622088706943
Model 3 - Fold:6 MSE: 5.865518277722259e+17  MAE: 9980189.442073422
Model 3 - Fold:7 MSE: 7.034037271202676e+21  MAE: 1175014454.4307468
Model 3 - Fold:8 MSE: 6.094967547253906e+22  MAE: 3089206777.4426227
Model 3 - Fold:9 MSE: 6.394159349336755e+21  MAE: 739754343.3615307
Model 3 - Fold:10 MSE: 2.1532573405301624e+21  MAE: 465218241.33396614


In [97]:
#Feature set 3
#new dataframe to outline error

df_diff = pd.DataFrame(list(zip(y_true_list, y_pred_list)))
df_diff.rename(columns={0: 'Actual', 1: 'Predicted'}, inplace=True)
df_diff['Error'] = df_diff['Predicted'] - df_diff['Actual']
total_len = len(df_diff)
df_diff

,Actual,Predicted,Error
0,90.0,19.141602,-70.858398
1,457.0,244.744141,-212.255859
2,330.0,282.805664,-47.194336
3,870.0,1140.140625,270.140625
4,110.0,-81.553711,-191.553711
...,...,...,...
117855,100.0,222.734375,122.734375
117856,1130.0,515.484375,-614.515625
117857,590.0,345.875000,-244.125000
117858,675.0,672.507812,-2.492188


In [99]:
diff_100_len = len(df_diff[(df_diff['Error'] > 100) | (df_diff['Error'] < -100)])
diff_100_len/total_len

0.6951467843203801

In [100]:
over_100_len = len(df_diff[(df_diff['Error'] > 100)])
over_100_len/total_len

0.4327676904802308

In [101]:
less_100_len = len(df_diff[(df_diff['Error'] < -100)])
less_100_len/total_len

0.26237909384014935

In [102]:
diff_1000_len = len(df_diff[(df_diff['Error'] > 1000) | (df_diff['Error'] < -1000)])
diff_1000_len/total_len

0.06289665705073816

In [103]:
diff_500_len = len(df_diff[(df_diff['Error'] > 500) | (df_diff['Error'] < -500)])
diff_500_len/total_len

0.1649244866791108

In [104]:
diff_300_len = len(df_diff[(df_diff['Error'] > 300) | (df_diff['Error'] < -300)])
diff_300_len/total_len

0.31366875954522316

In [76]:
#removing duration outliers (over 21 days)
dataframes_list = [x3, y3]
dataframes_list_transposed = [list_item.transpose() for list_item in dataframes_list]
merged_dataframe = pd.concat(dataframes_list_transposed, axis=0).transpose()
# merged_dataframe.dropna(subset=['Total Spending for transport'], inplace=True)
merged_dataframe.isnull().any().any()
merged_dataframe = merged_dataframe[merged_dataframe['DURATION'] <= 30]
y3 = merged_dataframe.loc[:,["SPD_DTOT"]]
x3 = merged_dataframe.loc[:, merged_dataframe.columns != "SPD_DTOT"]

In [77]:
y_true_list = []
y_pred_list = []

cnt = 1
for train_index, test_index in kf.split(x3, y3):
   X_train, X_test, y_train, y_test = train_test_split(x3, y3)
   model3 = LinearRegression().fit(X_train, y_train)
   y_pred = model3.predict(X_test)
   y_true_list = np.append(y_true_list, y_test)
   y_pred_list = np.append(y_pred_list,y_pred)
   mse_model3 = mean_squared_error(y_test, y_pred)
   mae_model = mean_absolute_error(y_test, y_pred)
   # print("Mean Score: " + str(np.mean(scores)))
   print(f"Model 3 - Fold:{cnt} MSE: {mse_model3}  MAE: {mae_model}")
   cnt += 1

Model 3 - Fold:1 MSE: 54714790530.39016  MAE: 2485.1450313005707
Model 3 - Fold:2 MSE: 6.768412451552222e+20  MAE: 649345699.6944681
Model 3 - Fold:3 MSE: 1.2625658876661786e+20  MAE: 106488236.46077049
Model 3 - Fold:4 MSE: 1.1696749787217658e+22  MAE: 1318237567.6649356
Model 3 - Fold:5 MSE: 2.1148978608109614e+20  MAE: 214759051.301795
Model 3 - Fold:6 MSE: 2.5231171595187507e+23  MAE: 11438372703.705019
Model 3 - Fold:7 MSE: 7.737544924439472e+18  MAE: 37230852.38536149
Model 3 - Fold:8 MSE: 1.174250327899853e+25  MAE: 63393811838.3768
Model 3 - Fold:9 MSE: 3.8183363377555833e+25  MAE: 62900117489.86504
Model 3 - Fold:10 MSE: 3.555747934158681e+20  MAE: 530917371.2830306


In [78]:
df_diff = pd.DataFrame(list(zip(y_true_list, y_pred_list)))
df_diff.rename(columns={0: 'Actual', 1: 'Predicted'}, inplace=True)
df_diff['Error'] = df_diff['Predicted'] - df_diff['Actual']
total_len = len(df_diff)
diff_100_len = len(df_diff[(df_diff['Error'] > 100) | (df_diff['Error'] < -100)])
diff_100_len/total_len

0.6924537943957073

In [111]:
#removing duration outliers (0 days)
dataframes_list = [x3, y3]
dataframes_list_transposed = [list_item.transpose() for list_item in dataframes_list]
merged_dataframe = pd.concat(dataframes_list_transposed, axis=0).transpose()
# merged_dataframe.dropna(subset=['Total Spending for transport'], inplace=True)
merged_dataframe.isnull().any().any()
#merged_dataframe[merged_dataframe['DURATION'] == 0]
merged_dataframe = merged_dataframe[merged_dataframe['DURATION'] > 0]
print(len(merged_dataframe))
y3 = merged_dataframe.loc[:,["SPD_DTOT"]]
x3 = merged_dataframe.loc[:, merged_dataframe.columns != "SPD_DTOT"]

21138


In [112]:
y_true_list = []
y_pred_list = []

cnt = 1
for train_index, test_index in kf.split(x3, y3):
   X_train, X_test, y_train, y_test = train_test_split(x3, y3)
   model3 = LinearRegression().fit(X_train, y_train)
   y_pred = model3.predict(X_test)
   y_true_list = np.append(y_true_list, y_test)
   y_pred_list = np.append(y_pred_list,y_pred)
   mse_model3 = mean_squared_error(y_test, y_pred)
   mae_model = mean_absolute_error(y_test, y_pred)
   # print("Mean Score: " + str(np.mean(scores)))
   print(f"Model 3 - Fold:{cnt} MSE: {mse_model3}  MAE: {mae_model}")
   cnt += 1

Model 3 - Fold:1 MSE: 5.101676946479526e+16  MAE: 3107514.0004937323
Model 3 - Fold:2 MSE: 4.029686114564392e+18  MAE: 47290098.624410555
Model 3 - Fold:3 MSE: 1.7451840278319383e+20  MAE: 229907315.14764664
Model 3 - Fold:4 MSE: 8.738233138229143e+19  MAE: 134332481.63630557
Model 3 - Fold:5 MSE: 1.5444613031448879e+24  MAE: 41038161521.49488
Model 3 - Fold:6 MSE: 1.3615247711996768e+17  MAE: 5076452.7777081365
Model 3 - Fold:7 MSE: 107973612.68904166  MAE: 764.9266763245033
Model 3 - Fold:8 MSE: 1.6671011360841747e+17  MAE: 5616954.180840528
Model 3 - Fold:9 MSE: 3.457311474948004e+24  MAE: 36173631378.12047
Model 3 - Fold:10 MSE: 67508219.42173345  MAE: 647.8160477767266


In [113]:
df_diff = pd.DataFrame(list(zip(y_true_list, y_pred_list)))
df_diff.rename(columns={0: 'Actual', 1: 'Predicted'}, inplace=True)
df_diff['Error'] = df_diff['Predicted'] - df_diff['Actual']
total_len = len(df_diff)
diff_100_len = len(df_diff[(df_diff['Error'] > 100) | (df_diff['Error'] < -100)])
diff_100_len/total_len

0.825600756859035

In [117]:
#removing duration outliers (total spend over $2024 - aligned with buckets found for categorization)
dataframes_list = [x3, y3]
dataframes_list_transposed = [list_item.transpose() for list_item in dataframes_list]
merged_dataframe = pd.concat(dataframes_list_transposed, axis=0).transpose()
# merged_dataframe.dropna(subset=['Total Spending for transport'], inplace=True)
merged_dataframe.isnull().any().any()
# merged_dataframe[merged_dataframe['SPD_DTOT'] < 2024]
merged_dataframe = merged_dataframe[merged_dataframe['SPD_DTOT'] < 2024]
print(len(merged_dataframe))
y3 = merged_dataframe.loc[:,["SPD_DTOT"]]
x3 = merged_dataframe.loc[:, merged_dataframe.columns != "SPD_DTOT"]

44783


In [118]:
y_true_list = []
y_pred_list = []

cnt = 1
for train_index, test_index in kf.split(x3, y3):
   X_train, X_test, y_train, y_test = train_test_split(x3, y3)
   model3 = LinearRegression().fit(X_train, y_train)
   y_pred = model3.predict(X_test)
   y_true_list = np.append(y_true_list, y_test)
   y_pred_list = np.append(y_pred_list,y_pred)
   mse_model3 = mean_squared_error(y_test, y_pred)
   mae_model = mean_absolute_error(y_test, y_pred)
   # print("Mean Score: " + str(np.mean(scores)))
   print(f"Model 3 - Fold:{cnt} MSE: {mse_model3}  MAE: {mae_model}")
   cnt += 1

Model 3 - Fold:1 MSE: 1.240908930454059e+22  MAE: 1690877289.1011271
Model 3 - Fold:2 MSE: 7.374968986871504e+21  MAE: 1318669455.45673
Model 3 - Fold:3 MSE: 1.6211187958890424e+19  MAE: 70068136.53127791
Model 3 - Fold:4 MSE: 2.7547886360083566e+23  MAE: 6312335595.585029
Model 3 - Fold:5 MSE: 5.389617940871911e+22  MAE: 4878267339.381677
Model 3 - Fold:6 MSE: 5.359839067338121e+25  MAE: 119816603330.03291
Model 3 - Fold:7 MSE: 3.168320100297372e+21  MAE: 1070998518.490979
Model 3 - Fold:8 MSE: 3.219555868330645e+19  MAE: 145461534.32934475
Model 3 - Fold:9 MSE: 6.062498689221081e+20  MAE: 504068578.63016367
Model 3 - Fold:10 MSE: 6.462936989148443e+21  MAE: 1551677887.0892954


In [122]:
df_diff = pd.DataFrame(list(zip(y_true_list, y_pred_list)))
df_diff.rename(columns={0: 'Actual', 1: 'Predicted'}, inplace=True)
df_diff['Error Ratio'] = df_diff['Predicted']/df_diff['Actual']
df_diff
# total_len = len(df_diff)
diff_100_len = len(df_diff[(df_diff['Error Ratio'] > 1.5) | (df_diff['Error Ratio'] < 0.5)])
diff_100_len/total_len

0.59978563772776

In [190]:
# feature set 4 - total spend

#consists of numerical inputs + ori dest new region, transport, transport, quarter, mainreason, high corr activities
x4_list = [num_features, ori_dest_metroarea, transport, quarter, main_reason, high_correlation_activities_cols]
x4_list_transposed = [list_item.transpose() for list_item in x4_list]
x4 = pd.concat(x4_list_transposed, axis=0).transpose()
y4 = total_spend

kf = KFold(n_splits=10,random_state=42,shuffle=True)

cnt = 1
for train_index, test_index in kf.split(x4, y4):
   X_train, X_test, y_train, y_test = train_test_split(x4, y4)
   model4 = LinearRegression().fit(X_train, y_train)
   y_pred = model4.predict(X_test)
   mse_model4 = mean_squared_error(y_test, y_pred)
   # print("Mean Score: " + str(np.mean(scores)))
   print(f"Model 4 - Fold:{cnt} MSE: {mse_model4}")
   cnt += 1

Model 4 - Fold:1 MSE: 1.853628605419901e+23
Model 4 - Fold:2 MSE: 9.993903129217383e+24
Model 4 - Fold:3 MSE: 3.326240847244027e+25
Model 4 - Fold:4 MSE: 2.4445352759385236e+24
Model 4 - Fold:5 MSE: 1.5417783776365887e+24
Model 4 - Fold:6 MSE: 1.1562668195167686e+21
Model 4 - Fold:7 MSE: 3.534296195309673e+24
Model 4 - Fold:8 MSE: 4.111871177004018e+22
Model 4 - Fold:9 MSE: 2.1519442316755426e+22
Model 4 - Fold:10 MSE: 7.46713479280081e+19


In [226]:
# feature set 3 - transport_spend

#consists of numerical inputs + ori dest new region, transport, transport, quarter
x3a_list = [num_features, ori_dest_newregion, transport, quarter, high_correlation_activities_cols, main_reason]
x3a_list_transposed = [list_item.transpose() for list_item in x3a_list]
x3a = pd.concat(x3a_list_transposed, axis=0).transpose()
y3a = transport_spend
dataframes_list = [x3a, y3a]
dataframes_list_transposed = [list_item.transpose() for list_item in dataframes_list]
merged_dataframe = pd.concat(dataframes_list_transposed, axis=0).transpose()
merged_dataframe.dropna(subset=['Total Spending for transport'], inplace=True)
merged_dataframe.isnull().any().any()
y3a = merged_dataframe.loc[:,["Total Spending for transport"]]
x3a = merged_dataframe.loc[:, merged_dataframe.columns != "Total Spending for transport"]

,DURATION,GNCQ06A,GNCQ06B,Origin Destination New Region_AlbertaCalgary Area,Origin Destination New Region_AlbertaCentral Alberta,Origin Destination New Region_AlbertaCentral Saskatchewan,Origin Destination New Region_AlbertaEastern Ontario,Origin Destination New Region_AlbertaEdmonton Area,Origin Destination New Region_AlbertaGTA,Origin Destination New Region_AlbertaHamilton-Niagara,...,Shopping,Sightseeing,Museum/Art Gallery,Historic site,Zoo/Aquarium,Wildife Viewing/Bird Watching,National/Provincial/Nature Park,Main Reason_Friends/Relatives,Main Reason_Holiday/Leisure/Recreation,Main Reason_Shopping
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,76.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47136,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
47137,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
47138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
47139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [230]:
kf = KFold(n_splits=10,random_state=42,shuffle=True)

cnt = 1
for train_index, test_index in kf.split(x3a, y3a):
   X_train, X_test, y_train, y_test = train_test_split(x3a, y3a)
   model3 = LinearRegression().fit(X_train, y_train)
   y_pred = model3.predict(X_test)
   mse_model = mean_squared_error(y_test, y_pred)
   # print("Mean Score: " + str(np.mean(scores)))
   print(f"Model 3a - Fold:{cnt} MSE: {mse_model}")
   cnt += 1

Model 3a - Fold:1 MSE: 5.5839143045203645e+23
Model 3a - Fold:2 MSE: 1.6901040515900775e+23
Model 3a - Fold:3 MSE: 7.5175908973612475e+22
Model 3a - Fold:4 MSE: 1.0636684563988823e+21
Model 3a - Fold:5 MSE: 8.88870392188904e+21
Model 3a - Fold:6 MSE: 7.108967483444261e+24
Model 3a - Fold:7 MSE: 8.716802805340117e+21
Model 3a - Fold:8 MSE: 2475554218394481.0
Model 3a - Fold:9 MSE: 5956957.163798673
Model 3a - Fold:10 MSE: 2.170219622518931e+18


In [228]:
# feature set 3 - food/bevs spend

#consists of numerical inputs + ori dest new region, transport, transport, quarter
x3b_list = [num_features, ori_dest_newregion, transport, quarter, high_correlation_activities_cols, main_reason]
x3b_list_transposed = [list_item.transpose() for list_item in x3b_list]
x3b = pd.concat(x3b_list_transposed, axis=0).transpose()
y3b = foodbev_spend
dataframes_list = [x3b, y3b]
dataframes_list_transposed = [list_item.transpose() for list_item in dataframes_list]
merged_dataframe = pd.concat(dataframes_list_transposed, axis=0).transpose()
merged_dataframe.dropna(subset=["Total Spending for food/bevs"], inplace=True)
merged_dataframe.isnull().any().any()
y3b = merged_dataframe.loc[:,["Total Spending for food/bevs"]]
x3b = merged_dataframe.loc[:, merged_dataframe.columns != "Total Spending for food/bevs"]

In [245]:
kf = KFold(n_splits=10,random_state=42,shuffle=True)


cnt = 1
for train_index, test_index in kf.split(x3b, y3b):
   X_train, X_test, y_train, y_test = train_test_split(x3b, y3b)
   model3 = LinearRegression().fit(X_train, y_train)
   y_pred = model3.predict(X_test)
   mse_model = mean_squared_error(y_test, y_pred)
   mae_model = mean_absolute_error(y_test, y_pred)
   print(f"Model 3a - Fold:{cnt} MSE: {mse_model} MAE: {mae_model}")
   cnt += 1


Model 3a - Fold:1 MSE: 3.5932274846369567e+18 MAE: 19586197.585756533
Model 3a - Fold:2 MSE: 1.1736567618116747e+19 MAE: 38257656.9707615
Model 3a - Fold:3 MSE: 3.971266047609044e+20 MAE: 233507390.01450542
Model 3a - Fold:4 MSE: 1.2344644313390158e+19 MAE: 46829874.020243034
Model 3a - Fold:5 MSE: 2.2409942272795198e+21 MAE: 489100046.009367
Model 3a - Fold:6 MSE: 3.350670243099086e+20 MAE: 198170022.94838595
Model 3a - Fold:7 MSE: 7.207558622918826e+20 MAE: 279070936.06091046
Model 3a - Fold:8 MSE: 1.280739405446636e+21 MAE: 632303436.6268955
Model 3a - Fold:9 MSE: 3.978260061719959e+20 MAE: 231425173.2342791
Model 3a - Fold:10 MSE: 1.4489244014144879e+20 MAE: 220930453.62066603


Check categorization code for method of tracking y pred and t actuals in two arrays 
Make into dataframe 
Add column that is the difference between the two 
Calculate number of differences over 1000, over 100, etc 
Try to figure out better error measure since not super accurate to only look at mse 